In [ ]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [3]:
cd ..

/home/elicer


In [4]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [5]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [6]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2
category = preprocess.Categorization

# Main

In [7]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner"], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
#         {"instance": basic, "method": ['customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level', 'job_function', 'customer_type'], "params": pkls.get("customer2")},
        {"instance": category, "method": ['cat_customer_idx','customer_idx_merge_enterprise','historical_cnt_mean'], "params": False},
    ]
}

## preprocessing

In [29]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [30]:
# prev column도 영향이 그렇게 많지는 않다. 뺴고 historical 그대로 사용하는 방법..?

cond = df_train['historical_existing_cnt']> 0 
df_train.loc[cond,"prev"] = 1
df_train['prev'] = df_train['prev'].fillna(0)

cond = df_test['historical_existing_cnt']> 0 
df_test.loc[cond,"prev"] = 1
df_test['prev'] = df_test['prev'].fillna(0)

## encoding

In [31]:
encode = Encode()

In [32]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
                        #'response_corporate',#"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1',
                     'new_customer_type1','new_customer_type2' # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [33]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [34]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [35]:
# label encoding
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [36]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro','ver_cus',
            'customer_country.1',
            "customer_country",#'response_corporate' # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'lead_owner','prev','historial_existing_mean','response_corporate',#'new_customer_idx'
            'customer_type2',
#             'customer_idx_converted_ratio', # ratio
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [37]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio']])
df_test[['lead_owner_converted_ratio']] = scaler.transform(df_test[['lead_owner_converted_ratio']])

## train

In [39]:
dict_ = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
model = _model.decision_tree
model.set_params(**dict_)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

In [144]:
model = _model.xgboost
model.set_params(random_state =42)
df_train['new_customer_idx'] = df_train['new_customer_idx'].astype(int)
df_test['new_customer_idx'] = df_test['new_customer_idx'].astype(int)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [263]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [244]:
# oversampling
from imblearn.over_sampling import SMOTE, ADASYN
adasyn = SMOTE(random_state=42)
X, Y = adasyn.fit_resample(X, Y)

In [264]:
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
train_result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
models = [] # 모델을 저장할 리스트

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 훈련
    clf = model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)

    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)

    score_result = check_the_score(predict, y_test)
    for name, score in score_result.items():
        result[name].append(score)

print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:02,  2.30it/s]

----[K-Fold Train Score]-----
f1 score : 0.9403 / STD: (+/- 0.0044)
precision score : 0.9104 / STD: (+/- 0.0071)
recall score : 0.9722 / STD: (+/- 0.0021)
roc_auc_score score : 0.9821 / STD: (+/- 0.0013)
----[K-Fold Validation Score]-----
f1 score : 0.8245 / STD: (+/- 0.0057)
precision score : 0.7745 / STD: (+/- 0.0113)
recall score : 0.8816 / STD: (+/- 0.0053)
roc_auc_score score : 0.9307 / STD: (+/- 0.0025)


In [274]:
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

lead_owner_converted_ratio: 0.2824322581291199
new_customer_idx: 0.14127878844738007
ver_win_rate_x: 0.07063854485750198
new_customer_type1: 0.05618283897638321
business_subarea: 0.03871985524892807
new_customer_type2: 0.03473649173974991
country: 0.02543959580361843
continent: 0.02530229650437832
com_reg_ver_win_rate: 0.024094238877296448
customer_interest: 0.023178208619356155
lead_desc_length: 0.022761009633541107
enterprise: 0.02275334671139717
historical_existing_cnt: 0.02217988669872284
business_area: 0.022108670324087143
new_inquiry_type: 0.02080654166638851
new_expected_timeline: 0.01947176642715931
business_unit: 0.019049091264605522
ver_win_ratio_per_bu: 0.017975009977817535
category_3: 0.01796533726155758
job_function: 0.017308568581938744
category_1: 0.01646765135228634
category_2: 0.01558714173734188
seniority_level: 0.015135696157813072
bant_submit: 0.015129214152693748
less_timeline: 0.013297954574227333


## test & submission

In [267]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [266]:
# predict
pred = model.predict(test_drop)
pred = np.where(pred == 0, False, True)

In [156]:
y_pred_proba

array([[8.4236425e-01, 1.5763575e-01],
       [9.9877137e-01, 1.2286216e-03],
       [9.8365396e-01, 1.6346015e-02],
       ...,
       [9.9989688e-01, 1.0309268e-04],
       [9.9997801e-01, 2.2006123e-05],
       [9.9985266e-01, 1.4731596e-04]], dtype=float32)

In [276]:
# threshold !!
y_pred_proba = model.predict_proba(test_drop)
empty = np.empty(y_pred_proba.shape[0], dtype=bool) # 새 빈 배열 만들기
threshold = 0.9
for idx, val in enumerate(y_pred_proba):
    if np.max(val) < threshold: # 배열의 요소의 max가 0.9보다 작으면 무조건 True
        empty[idx] = True
    else:
        if np.argmax(val): # idx:1(True) 이면 True
            empty[idx] = True
        else: # idx:0(False) 이면 False
            empty[idx] = False
print(sum(empty))
pred = empty

1404


In [257]:
pred1 = pred  #pred1 718 | pred2 1142 | pred3 820

In [259]:
su = 0
for i,j in zip(pred2, pred1):
    if i != j:
        su +=1
su

378

In [ ]:
# pred1_ = pred1.reshape(-1,1)
# pred2_ = pred2.reshape(-1,1)
# pred3_ = pred3.reshape(-1,1)
# combine = np.hstack((pred1_, pred2_, pred3_))
# combine_ = np.round(combine.mean(axis=1))

In [49]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)